In [1]:
#import statements
import numpy as np
from PIL import Image
%matplotlib inline

import cv2

from pathlib import Path

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'    # Suppress TensorFlow logging (1)
import pathlib
import tensorflow as tf

tf.get_logger().setLevel('ERROR')           # Suppress TensorFlow logging (2)

import time
import bbox_visualizer as bbv
#img_bbox = bbv.draw_rectangle(img_arr, [427, 347, 1278, 672])

import pandas as pd

### STEPS
1. read annotations.csv (format - (xmin, ymin, width, height))
2. convert the format to (ymin, xmin, xmax, ymax)
3. store the updated csv to the location

In [4]:
csv_path = Path("../../../dataset/object_detection/annotations/update_csv/new_images.csv")

In [5]:
annotations = pd.read_csv(csv_path)

In [7]:
annotations.head()

,label,xmin,ymin,width_box,height_box,image_name,width_image,height_image
0,bowl,59,18,49,46,image_01.jpg,275,183
1,bowl,126,17,47,44,image_01.jpg,275,183
2,bowl,59,70,82,73,image_01.jpg,275,183
3,bowl,213,80,48,44,image_01.jpg,275,183
4,bowl,147,69,46,47,image_01.jpg,275,183


In [8]:
annotations["xmax"] = annotations["xmin"] + annotations["width_box"]
annotations["ymax"] = annotations["ymin"] + annotations["height_box"]

In [9]:
annotations.head()

,label,xmin,ymin,width_box,height_box,image_name,width_image,height_image,xmax,ymax
0,bowl,59,18,49,46,image_01.jpg,275,183,108,64
1,bowl,126,17,47,44,image_01.jpg,275,183,173,61
2,bowl,59,70,82,73,image_01.jpg,275,183,141,143
3,bowl,213,80,48,44,image_01.jpg,275,183,261,124
4,bowl,147,69,46,47,image_01.jpg,275,183,193,116


In [10]:
reordered_cols = ['image_name','label','ymin','xmin','ymax','xmax','width_box','height_box','width_image','height_image']

In [11]:
annotations = annotations[reordered_cols]

In [12]:
annotations.head()

,image_name,label,ymin,xmin,ymax,xmax,width_box,height_box,width_image,height_image
0,image_01.jpg,bowl,18,59,64,108,49,46,275,183
1,image_01.jpg,bowl,17,126,61,173,47,44,275,183
2,image_01.jpg,bowl,70,59,143,141,82,73,275,183
3,image_01.jpg,bowl,80,213,124,261,48,44,275,183
4,image_01.jpg,bowl,69,147,116,193,46,47,275,183


In [13]:
annotations.to_csv(csv_path, index=False)

### Remove old images and replace new ones annotations

In [2]:
new_annotations_path = Path("../../../dataset/object_detection/annotations/update_csv/bed_annotations.csv")
annotations_path = Path("../../../dataset/object_detection/annotations/update_csv/annotations.csv")

In [3]:
new_annotations = pd.read_csv(new_annotations_path)
annotations = pd.read_csv(annotations_path)

In [11]:
new_annotations.head()

,label,xmin,ymin,width_box,height_box,image_name,width_image,height_image,xmax,ymax
0,bed,175,458,759,285,image_127.png,934,743,934,743
1,potted plant,445,261,166,174,image_127.png,934,743,611,435
2,bed,179,1612,1693,1630,image_128.jpg,3024,3809,1872,3242
3,potted plant,642,867,313,593,image_128.jpg,3024,3809,955,1460
4,laptop,1013,1990,397,281,image_128.jpg,3024,3809,1410,2271


In [5]:
annotations.head()

,image_name,label,ymin,xmin,ymax,xmax,width_box,height_box,width_image,height_image,sort_column
0,image_01.jpg,bowl,18,59,64.0,108.0,49,46,275,183,1
1,image_01.jpg,bowl,31,173,76.0,219.0,46,45,275,183,1
2,image_01.jpg,bowl,69,147,116.0,193.0,46,47,275,183,1
3,image_01.jpg,bowl,17,126,61.0,173.0,47,44,275,183,1
4,image_01.jpg,bowl,70,59,143.0,141.0,82,73,275,183,1


In [6]:
images_to_replace = list(set(new_annotations["image_name"]))
len(images_to_replace)

23

In [7]:
print("Total annotations before replace: {0}".format(len(annotations)))

Total annotations before replace: 2550


In [90]:
print("New annotations counts: {0}".format(len(new_annotations)))

New annotations counts: 93


In [8]:
annotations_without_old_imgs = annotations[~annotations["image_name"].isin(images_to_replace)]

In [9]:
print("annotations without removed images which are going to be replaced is: {0}".format(len(annotations_without_old_imgs)))

annotations without removed images which are going to be replaced is: 2457


In [10]:
new_annotations["xmax"] = new_annotations["xmin"] + new_annotations["width_box"]
new_annotations["ymax"] = new_annotations["ymin"] + new_annotations["height_box"]

In [13]:
reordered_cols = ['image_name','label','ymin','xmin','ymax','xmax','width_box','height_box','width_image','height_image']

In [14]:
new_annotations = new_annotations[reordered_cols]
new_annotations.head()

,image_name,label,ymin,xmin,ymax,xmax,width_box,height_box,width_image,height_image
0,image_127.png,bed,458,175,743,934,759,285,934,743
1,image_127.png,potted plant,261,445,435,611,166,174,934,743
2,image_128.jpg,bed,1612,179,3242,1872,1693,1630,3024,3809
3,image_128.jpg,potted plant,867,642,1460,955,313,593,3024,3809
4,image_128.jpg,laptop,1990,1013,2271,1410,397,281,3024,3809


In [15]:
frames = [annotations_without_old_imgs, new_annotations]
final_annotations = pd.concat(frames)

In [16]:
print("final number of annotated objects present in our dataset: {0}".format(len(final_annotations)))

final number of annotated objects present in our dataset: 2550


In [17]:
final_annotations.head()

,image_name,label,ymin,xmin,ymax,xmax,width_box,height_box,width_image,height_image,sort_column
0,image_01.jpg,bowl,18,59,64.0,108.0,49,46,275,183,1.0
1,image_01.jpg,bowl,31,173,76.0,219.0,46,45,275,183,1.0
2,image_01.jpg,bowl,69,147,116.0,193.0,46,47,275,183,1.0
3,image_01.jpg,bowl,17,126,61.0,173.0,47,44,275,183,1.0
4,image_01.jpg,bowl,70,59,143.0,141.0,82,73,275,183,1.0


In [18]:
def strip_number(image_name):
    start_index = image_name.index("_")
    end_index = image_name.index(".")
    return int(image_name[start_index+1: end_index])

In [19]:
final_annotations["sort_column"] = final_annotations["image_name"].apply(strip_number)
final_annotations.head()

,image_name,label,ymin,xmin,ymax,xmax,width_box,height_box,width_image,height_image,sort_column
0,image_01.jpg,bowl,18,59,64.0,108.0,49,46,275,183,1
1,image_01.jpg,bowl,31,173,76.0,219.0,46,45,275,183,1
2,image_01.jpg,bowl,69,147,116.0,193.0,46,47,275,183,1
3,image_01.jpg,bowl,17,126,61.0,173.0,47,44,275,183,1
4,image_01.jpg,bowl,70,59,143.0,141.0,82,73,275,183,1


In [20]:
final_annotations.sort_values(by=["sort_column"], inplace=True)

In [21]:
final_annotations.tail()

,image_name,label,ymin,xmin,ymax,xmax,width_box,height_box,width_image,height_image,sort_column
2545,image_996.jpg,tv,64,395,219.0,527.0,132,155,670,335,996
2546,image_997.jpg,tv,113,297,227.0,442.0,145,114,550,330,997
2547,image_998.jpg,tv,145,666,324.0,951.0,285,179,1280,720,998
2548,image_999.jpg,tv,184,220,357.0,349.0,129,173,900,805,999
2549,image_1000.jpg,tv,145,131,323.0,422.0,291,178,1024,683,1000


In [26]:
final_annotations["ymax"] = final_annotations["ymax"].apply(int)
final_annotations["xmax"] = final_annotations["xmax"].apply(int)

In [27]:
final_annotations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2550 entries, 0 to 2549
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   image_name    2550 non-null   object
 1   label         2550 non-null   object
 2   ymin          2550 non-null   int64 
 3   xmin          2550 non-null   int64 
 4   ymax          2550 non-null   int64 
 5   xmax          2550 non-null   int64 
 6   width_box     2550 non-null   int64 
 7   height_box    2550 non-null   int64 
 8   width_image   2550 non-null   int64 
 9   height_image  2550 non-null   int64 
 10  sort_column   2550 non-null   int64 
dtypes: int64(9), object(2)
memory usage: 239.1+ KB


In [28]:
final_annotations.to_csv(annotations_path, index=False)

In [29]:
len(list((final_annotations["sort_column"])))

2550

In [30]:
len(list(set(final_annotations["sort_column"])))

1000